# 📊 Análise GTFS - SPTrans

Este notebook realiza uma análise exploratória completa dos dados GTFS (General Transit Feed Specification) da SPTrans.

## Objetivos
1. Validar integridade dos arquivos GTFS
2. Analisar distribuição de rotas, paradas e viagens
3. Identificar padrões temporais e espaciais
4. Detectar anomalias e inconsistências
5. Gerar insights sobre a malha de transporte

---

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import warnings
from pathlib import Path
from datetime import datetime, timedelta
import json

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Bibliotecas importadas com sucesso!")

## 1️⃣ Carregamento dos Dados GTFS

In [ ]:
# Diretório GTFS
GTFS_PATH = Path('../data/gtfs/')

# Verificar se diretório existe
if not GTFS_PATH.exists():
    print(f"⚠️ Diretório {GTFS_PATH} não encontrado!")
    print("Por favor, baixe os arquivos GTFS primeiro.")
else:
    print(f"✅ Diretório GTFS encontrado: {GTFS_PATH}")
    print(f"\nArquivos disponíveis:")
    for file in sorted(GTFS_PATH.glob('*.txt')):
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"  - {file.name:20s} ({size_mb:.2f} MB)")

In [ ]:
# Função para carregar GTFS
def load_gtfs_files(gtfs_path):
    """Carrega todos os arquivos GTFS em um dicionário de DataFrames"""
    gtfs_files = {
        'routes': 'routes.txt',
        'trips': 'trips.txt',
        'stops': 'stops.txt',
        'stop_times': 'stop_times.txt',
        'shapes': 'shapes.txt',
        'calendar': 'calendar.txt',
        'calendar_dates': 'calendar_dates.txt',
        'agency': 'agency.txt'
    }
    
    gtfs_data = {}
    
    for name, filename in gtfs_files.items():
        file_path = gtfs_path / filename
        if file_path.exists():
            try:
                df = pd.read_csv(file_path, low_memory=False)
                gtfs_data[name] = df
                print(f"✅ {filename:25s} - {len(df):,} linhas")
            except Exception as e:
                print(f"❌ Erro ao carregar {filename}: {e}")
        else:
            print(f"⚠️ {filename:25s} - NÃO ENCONTRADO")
    
    return gtfs_data

# Carregar dados
print("📥 Carregando arquivos GTFS...\n")
gtfs = load_gtfs_files(GTFS_PATH)
print(f"\n✅ {len(gtfs)} arquivos carregados com sucesso!")

## 2️⃣ Visão Geral dos Dados

In [ ]:
# Resumo geral
print("📊 RESUMO GERAL DOS DADOS GTFS")
print("=" * 60)

if 'routes' in gtfs:
    print(f"🚌 Rotas (routes):        {len(gtfs['routes']):>10,}")

if 'trips' in gtfs:
    print(f"🛣️  Viagens (trips):       {len(gtfs['trips']):>10,}")

if 'stops' in gtfs:
    print(f"🚏 Paradas (stops):       {len(gtfs['stops']):>10,}")

if 'stop_times' in gtfs:
    print(f"⏰ Horários (stop_times): {len(gtfs['stop_times']):>10,}")

if 'shapes' in gtfs:
    print(f"🗺️  Formas (shapes):       {len(gtfs['shapes']):>10,} pontos")

print("=" * 60)

In [ ]:
# Estrutura de cada arquivo
for name, df in gtfs.items():
    print(f"\n📋 {name.upper()}")
    print("-" * 60)
    print(f"Colunas: {list(df.columns)}")
    print(f"Tipos: {df.dtypes.to_dict()}")
    print(f"Shape: {df.shape}")

## 3️⃣ Análise de Rotas (Routes)

In [ ]:
if 'routes' in gtfs:
    routes = gtfs['routes']
    
    print("🚌 ANÁLISE DE ROTAS")
    print("=" * 60)
    
    # Amostra
    display(routes.head(10))
    
    # Estatísticas
    print(f"\n📊 Estatísticas:")
    print(f"  - Total de rotas: {len(routes):,}")
    print(f"  - Rotas únicas: {routes['route_id'].nunique():,}")
    
    if 'route_type' in routes.columns:
        print(f"\n🚦 Distribuição por tipo de rota:")
        route_types = routes['route_type'].value_counts()
        for rt, count in route_types.items():
            print(f"  - Tipo {rt}: {count:,} rotas")

In [ ]:
if 'routes' in gtfs:
    # Gráfico de distribuição de tipos de rotas
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Gráfico de barras
    if 'route_type' in routes.columns:
        route_type_counts = routes['route_type'].value_counts()
        axes[0].bar(route_type_counts.index.astype(str), route_type_counts.values, color='steelblue')
        axes[0].set_title('Distribuição de Rotas por Tipo', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Tipo de Rota')
        axes[0].set_ylabel('Quantidade')
        axes[0].grid(axis='y', alpha=0.3)
    
    # Top 20 rotas por nome
    if 'route_short_name' in routes.columns:
        top_routes = routes['route_short_name'].value_counts().head(20)
        axes[1].barh(range(len(top_routes)), top_routes.values, color='coral')
        axes[1].set_yticks(range(len(top_routes)))
        axes[1].set_yticklabels(top_routes.index)
        axes[1].set_title('Top 20 Rotas Mais Comuns', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Quantidade')
        axes[1].invert_yaxis()
        axes[1].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4️⃣ Análise de Paradas (Stops)

In [ ]:
if 'stops' in gtfs:
    stops = gtfs['stops']
    
    print("🚏 ANÁLISE DE PARADAS")
    print("=" * 60)
    
    # Amostra
    display(stops.head(10))
    
    # Estatísticas geográficas
    if 'stop_lat' in stops.columns and 'stop_lon' in stops.columns:
        print(f"\n📍 Estatísticas Geográficas:")
        print(f"  - Latitude mínima:  {stops['stop_lat'].min():.6f}")
        print(f"  - Latitude máxima:  {stops['stop_lat'].max():.6f}")
        print(f"  - Longitude mínima: {stops['stop_lon'].min():.6f}")
        print(f"  - Longitude máxima: {stops['stop_lon'].max():.6f}")
        print(f"  - Centro aproximado: ({stops['stop_lat'].mean():.6f}, {stops['stop_lon'].mean():.6f})")

In [ ]:
if 'stops' in gtfs and 'stop_lat' in stops.columns:
    # Mapa de calor das paradas
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribuição de latitude e longitude
    axes[0].hist(stops['stop_lat'], bins=50, alpha=0.7, color='blue', label='Latitude')
    axes[0].set_title('Distribuição de Latitude das Paradas', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Latitude')
    axes[0].set_ylabel('Frequência')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    axes[1].hist(stops['stop_lon'], bins=50, alpha=0.7, color='red', label='Longitude')
    axes[1].set_title('Distribuição de Longitude das Paradas', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Longitude')
    axes[1].set_ylabel('Frequência')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
if 'stops' in gtfs and 'stop_lat' in stops.columns:
    # Mapa interativo com Folium
    print("🗺️ Gerando mapa interativo das paradas...")
    
    # Centro de São Paulo
    center_lat = stops['stop_lat'].mean()
    center_lon = stops['stop_lon'].mean()
    
    # Criar mapa
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Adicionar mapa de calor (sample de 5000 pontos para performance)
    heat_data = stops[['stop_lat', 'stop_lon']].sample(min(5000, len(stops))).values.tolist()
    plugins.HeatMap(heat_data, radius=15, blur=25).add_to(m)
    
    # Adicionar algumas paradas como marcadores (sample de 50)
    sample_stops = stops.sample(min(50, len(stops)))
    for _, stop in sample_stops.iterrows():
        folium.CircleMarker(
            location=[stop['stop_lat'], stop['stop_lon']],
            radius=3,
            popup=stop.get('stop_name', 'Parada'),
            color='blue',
            fill=True,
            fillColor='blue'
        ).add_to(m)
    
    # Salvar mapa
    map_path = '../data/samples/stops_heatmap.html'
    m.save(map_path)
    print(f"✅ Mapa salvo em: {map_path}")
    
    # Exibir mapa
    display(m)

## 5️⃣ Análise de Viagens (Trips)

In [ ]:
if 'trips' in gtfs:
    trips = gtfs['trips']
    
    print("🛣️ ANÁLISE DE VIAGENS")
    print("=" * 60)
    
    # Amostra
    display(trips.head(10))
    
    # Estatísticas
    print(f"\n📊 Estatísticas:")
    print(f"  - Total de viagens: {len(trips):,}")
    print(f"  - Viagens únicas: {trips['trip_id'].nunique():,}")
    print(f"  - Rotas únicas: {trips['route_id'].nunique():,}")
    
    if 'service_id' in trips.columns:
        print(f"  - Serviços únicos: {trips['service_id'].nunique():,}")

In [ ]:
if 'trips' in gtfs and 'routes' in gtfs:
    # Juntar trips com routes
    trips_routes = trips.merge(routes, on='route_id', how='left')
    
    # Top 20 rotas com mais viagens
    top_routes_trips = trips_routes.groupby('route_short_name').size().sort_values(ascending=False).head(20)
    
    plt.figure(figsize=(14, 8))
    plt.barh(range(len(top_routes_trips)), top_routes_trips.values, color='teal')
    plt.yticks(range(len(top_routes_trips)), top_routes_trips.index)
    plt.title('Top 20 Rotas com Mais Viagens Programadas', fontsize=16, fontweight='bold')
    plt.xlabel('Número de Viagens')
    plt.ylabel('Rota')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6️⃣ Análise de Horários (Stop Times)

In [ ]:
if 'stop_times' in gtfs:
    stop_times = gtfs['stop_times']
    
    print("⏰ ANÁLISE DE HORÁRIOS")
    print("=" * 60)
    
    # Amostra
    display(stop_times.head(10))
    
    # Estatísticas
    print(f"\n📊 Estatísticas:")
    print(f"  - Total de horários: {len(stop_times):,}")
    print(f"  - Viagens únicas: {stop_times['trip_id'].nunique():,}")
    print(f"  - Paradas únicas: {stop_times['stop_id'].nunique():,}")

In [ ]:
if 'stop_times' in gtfs:
    # Converter arrival_time para hora
    def parse_time(time_str):
        """Parse GTFS time format (HH:MM:SS) handling 24+ hours"""
        try:
            parts = time_str.split(':')
            hour = int(parts[0]) % 24  # Handle 24+ hours
            return hour
        except:
            return None
    
    if 'arrival_time' in stop_times.columns:
        stop_times['hour'] = stop_times['arrival_time'].apply(parse_time)
        
        # Distribuição de horários
        hourly_distribution = stop_times['hour'].value_counts().sort_index()
        
        plt.figure(figsize=(14, 6))
        plt.bar(hourly_distribution.index, hourly_distribution.values, color='purple', alpha=0.7)
        plt.title('Distribuição de Horários de Ônibus por Hora do Dia', fontsize=16, fontweight='bold')
        plt.xlabel('Hora do Dia')
        plt.ylabel('Número de Passagens')
        plt.xticks(range(24))
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print("\n🕐 Horários de pico:")
        top_hours = hourly_distribution.sort_values(ascending=False).head(5)
        for hour, count in top_hours.items():
            print(f"  - {hour:02d}:00h - {count:,} passagens")

## 7️⃣ Validação de Qualidade dos Dados

In [ ]:
print("✅ VALIDAÇÃO DE QUALIDADE DOS DADOS GTFS")
print("=" * 60)

quality_report = {}

for name, df in gtfs.items():
    print(f"\n📋 {name.upper()}:")
    
    # Valores nulos
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print(f"  ⚠️ Valores nulos encontrados:")
        for col, count in null_counts[null_counts > 0].items():
            pct = (count / len(df)) * 100
            print(f"    - {col}: {count:,} ({pct:.2f}%)")
    else:
        print(f"  ✅ Sem valores nulos")
    
    # Duplicatas
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"  ⚠️ Linhas duplicadas: {duplicates:,}")
    else:
        print(f"  ✅ Sem duplicatas")
    
    quality_report[name] = {
        'total_rows': len(df),
        'null_values': null_counts.sum(),
        'duplicates': duplicates
    }

print("\n" + "=" * 60)
print("✅ Validação concluída!")

## 8️⃣ Insights e Conclusões

In [ ]:
print("🎯 PRINCIPAIS INSIGHTS DA ANÁLISE GTFS")
print("=" * 60)

insights = []

if 'routes' in gtfs:
    total_routes = len(gtfs['routes'])
    insights.append(f"📊 Sistema conta com {total_routes:,} rotas de transporte")

if 'stops' in gtfs:
    total_stops = len(gtfs['stops'])
    insights.append(f"🚏 Total de {total_stops:,} paradas na malha de transporte")

if 'trips' in gtfs:
    total_trips = len(gtfs['trips'])
    insights.append(f"🛣️ {total_trips:,} viagens programadas no sistema")

if 'stop_times' in gtfs and 'hour' in stop_times.columns:
    peak_hour = stop_times['hour'].value_counts().idxmax()
    insights.append(f"🕐 Horário de pico: {peak_hour:02d}:00h")

for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\n" + "=" * 60)
print("✅ Análise GTFS concluída com sucesso!")
print("\n📌 Próximos passos:")
print("  1. Validar integridade referencial entre tabelas")
print("  2. Integrar com dados em tempo real da API")
print("  3. Criar visualizações avançadas de rotas")
print("  4. Analisar padrões temporais de utilização")

## 📝 Exportar Relatório

In [ ]:
# Salvar relatório em JSON
report = {
    'timestamp': datetime.now().isoformat(),
    'gtfs_summary': {name: {'rows': len(df), 'columns': len(df.columns)} for name, df in gtfs.items()},
    'quality_report': quality_report,
    'insights': insights
}

report_path = '../data/samples/gtfs_analysis_report.json'
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"✅ Relatório salvo em: {report_path}")